# Agents

## 1.静态模型ß

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool

model = init_chat_model(
    "openai:gpt-3.5-turbo",
    temperature=0
)

def get_weather(city: str) -> str:
  """Get weather for a given city."""
  return f"It's always sunny in {city}!"

tools=[get_weather],

In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    "openai:gpt-5",
    tools=[]
)

## 2.动态模型

In [ ]:
"""
创建一个智能体，它会在对话早期使用轻量、便宜的 gpt-4.1-mini，当对话变长时自动切换到更强大的 gpt-5。这样既能节省成本，又能保证复杂任务的质量。
"""
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent, AgentState
from langgraph.runtime import Runtime

def select_model(state: AgentState, runtime: Runtime) -> ChatOpenAI:
    """Choose model based on conversation complexity."""
    messages = state["messages"]
    message_count = len(messages)

    if message_count < 10:
        return ChatOpenAI(model="gpt-4.1-mini").bind_tools(tools)
    else:
        return ChatOpenAI(model="gpt-5").bind_tools(tools) # Better model for longer conversations

agent = create_agent(select_model, tools=tools)

## 3.工具

In [ ]:
from langchain_core.tools import tool
from langchain.agents import create_agent

@tool
def search(query: str) -> str:
    """Search for information."""
    return f"Results for: {query}"

@tool
def calculate(expression: str) -> str:
    """Perform calculations."""
    return str(eval(expression))

agent = create_agent(model, tools=[search, calculate])

## 3.1 传递配置好的 ToolNode

In [ ]:
from langchain.agents import ToolNode

tool_node = ToolNode(
    tools=[search, calculate],
    handle_tool_errors="Please check your input and try again."
)
agent = create_agent(model, tools=tool_node)
result = agent.invoke({"messages": [...]})

## 4.prompt

In [ ]:
agent = create_agent(
    model,
    tools,
    prompt="You are a helpful assistant. Be concise and accurate."
)

## 高级配置 -- 结构化输出

In [ ]:
from pydantic import BaseModel
from langchain.agents import create_agent

class ContactInfo(BaseModel):
    name: str
    email: str
    phone: str

agent = create_agent(
    model,
    tools=[search_tool],
    response_format=ContactInfo
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
})

result["structured_response"]
# ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

## 高级配置 -- 记忆

In [ ]:
from typing import TypedDict
from typing_extensions import Annotated
from langgraph.graph.message import add_messages
from langchain.agents import create_agent
from langchain.agents import AgentState

class CustomAgentState(AgentState):
    messages: Annotated[list, add_messages]
    user_preferences: dict

agent = create_agent(
    model,
    tools=tools,
    state_schema=CustomAgentState
)

# The agent can now track additional state beyond messages. This custom state can be accessed and updated throughout the conversation.
result = agent.invoke({
    "messages": [{"role": "user", "content": "I prefer technical explanations"}],
    "user_preferences": {"style": "technical", "verbosity": "detailed"},
})

## 高级配置 -- 模型前钩子

In [ ]:
from langchain_core.messages import RemoveMessage
from langgraph.graph.message import REMOVE_ALL_MESSAGES
from langchain.agents import create_agent

def trim_messages(state):
    """Keep only the last few messages to fit context window."""
    messages = state["messages"]

    if len(messages) <= 3:
        return {"messages": messages}

    first_msg = messages[0]
    recent_messages = messages[-3:] if len(messages) % 2 == 0 else messages[-4:]
    new_messages = [first_msg] + recent_messages

    return {
        "messages": [
            RemoveMessage(id=REMOVE_ALL_MESSAGES),
            *new_messages
        ]
    }

agent = create_agent(
    model,
    tools=tools,
    pre_model_hook=trim_messages
)

## 高级配置 --  模型后钩子

In [ ]:
from langchain_core.messages import AIMessage, RemoveMessage
from langgraph.graph.message import REMOVE_ALL_MESSAGES

def validate_response(state):
    """Check model response for policy violations."""
    messages = state["messages"]
    last_message = messages[-1]

    ## confidential 机密
    if "confidential" in last_message.content.lower():
        return {
            "messages": [
                RemoveMessage(id=REMOVE_ALL_MESSAGES),
                *messages[:-1],
                AIMessage(content="I cannot share confidential information.")
            ]
        }

    return {}

agent = create_agent(
    model,
    tools=tools,
    post_model_hook=validate_response
)

## 高级配置 -- 流式传输

In [ ]:
for chunk in agent.stream({
    "messages": [{"role": "user", "content": "Search for AI news and summarize the findings"}]
}, stream_mode="values"):
    # Each chunk contains the full state at that point
    latest_message = chunk["messages"][-1]
    if latest_message.content:
        print(f"Agent: {latest_message.content}")
    elif latest_message.tool_calls:
        print(f"Calling tools: {[tc['name'] for tc in latest_message.tool_calls]}")

In [5]:
## 流失输出 demo

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain.agents import create_agent


# 1. 定义一个最简单的工具
@tool
def search_ai_news(query: str) -> str:
    """Search the latest AI news (dummy implementation)."""
    return f"Found some dummy AI news for query: {query}"


# 2. 定义模型（这里用 OpenAI）
model = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# 3. 创建 agent
agent = create_agent(
    model=model,
    tools=[search_ai_news]
)

# 4. 运行并流式打印结果
for chunk in agent.stream(
    {
        "messages": [
            {"role": "user", "content": "Search for AI news and summarize the findings"}
        ]
    },
    stream_mode="values",
):
    # 每个 chunk 是当前状态
    latest_message = chunk["messages"][-1]
    if latest_message.content:
        print(f"Agent: {latest_message.content}")
    elif latest_message.tool_calls:
        print(f"Calling tools: {[tc['name'] for tc in latest_message.tool_calls]}")


Agent: Search for AI news and summarize the findings
Calling tools: ['search_ai_news']
Agent: Found some dummy AI news for query: latest AI news
Agent: Here is a summary of the latest AI news:

- There have been recent advancements in AI technologies, including improvements in natural language processing and computer vision.
- New AI models have been released that offer better performance and efficiency.
- AI is increasingly being integrated into various industries such as healthcare, finance, and autonomous vehicles.
- Ethical considerations and regulations around AI development and deployment continue to be a significant topic of discussion.
- Research in AI safety and interpretability is gaining more attention to ensure responsible AI use.

If you want, I can provide more detailed information on any specific area of AI news.
